# CSC 820 HW #5
## Khalid Mehtab Khan
## SFSU ID: 923673423

- All the comments and explanation in the notebook are added as text markdown


In [46]:
!pip install -U dill

In [3]:
!pip install -U nltk==3.4

  Using cached nltk-3.4-py3-none-any.whl
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [4]:
# Upgrading NLTK is necessary to fix errors related to padded sequences.
# This update ensures that all required imports work properly.

# if the session is restarted becuase of this, please upgrade nltk before moving ahead

!pip install --upgrade nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.4
    Uninstalling nltk-3.4:
      Successfully uninstalled nltk-3.4


---------------------------------------------------------------------------------------

Import necessary functions from NLTK for text preprocessing, including padding, and creating bigrams, ngrams, and everygrams for language modeling.

- 'pad_sequence' for adding start or end padding to word sequences.
- 'bigrams' for creating pairs of adjacent words.
- 'ngrams' for making groups of words up to a specified number.
- 'everygrams' for generating all possible word combinations.
- 'pad_both_ends' for adding specific markers to indicate the beginning and end of sequences.

In [5]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Import pandas for data manipulation, and NLTK's tokenization functions for splitting text into words and sentences.

In [7]:
import pandas as pd
from nltk import word_tokenize, sent_tokenize

Import language modeling tools from NLTK: 'padded_everygram_pipeline' for preparing text data, and 'MLE' for creating a language model using the Maximum Likelihood Estimation approach.

- 'padded_everygram_pipeline' for preparing text data with padding and generating n-grams for language modeling.
- 'MLE' from NLTK for creating a language model using the Maximum Likelihood Estimation method.

In [8]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

This example shows how the bigram function turns arrays of words into pairs called bigrams.

- Each pair is made from words in the same sentence, not from different sentences. This helps us understand the sentence better and improves our predictions for the next words.

We create two example sentences split into individual words. This setup helps us learn how tasks like breaking down text and forming word pairs work in language processing.

In [9]:
text = [['I','need','to','book', 'ticket', 'to', 'Australia' ], ['I', 'want', 'to' ,'read', 'a' ,'book', 'of' ,'Shakespeare']]

We can see in context of our padded language model sentences, that these bigrams are without "s" marking at the beginning and the '/s' end of the sentence because no preprocessing of the text has been done.

In [11]:
list(bigrams(text[0]))

[('I', 'need'),
 ('need', 'to'),
 ('to', 'book'),
 ('book', 'ticket'),
 ('ticket', 'to'),
 ('to', 'Australia')]

Trigrams can be used in a similar way

- Generate trigrams from the second sentence in our text. This is an example of how to produce n-grams of length three from a sequence of tokens

In [12]:
list(ngrams(text[1], n=3))

[('I', 'want', 'to'),
 ('want', 'to', 'read'),
 ('to', 'read', 'a'),
 ('read', 'a', 'book'),
 ('a', 'book', 'of'),
 ('book', 'of', 'Shakespeare')]

- As this project was setup on google colab, the dataset was upload on the drive, if data is used in any other way please change the next two blocks of code.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
trump_tweet_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/realdonaldtrump.csv')
trump_tweet_data.head()

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


In [29]:
# using the columns method of data frame, we can anlayse what types of information the data frame provides
trump_tweet_data.columns

Index(['id', 'link', 'content', 'date', 'retweets', 'favorites', 'mentions',
       'hashtags'],
      dtype='object')

Importing Dataset from drive

- Here we load a dataset of Donald Trump's tweets using pandas, which is a tool for data manipulation. We then display the first few rows of the dataset with '.head()' to see what the data looks like and identify the columns, especially where the tweets are stored

In the data frame, each row represents a tweet. The dataframe provides more infromation aeach tweets which can be seen as the columns of the data.

Each tweet has -
  'id',
  'link',
  'content',
  'date',
  'retweets',
  'favorites',
  'mentions',
  'hashtags'

In [30]:
trump_tokens = list(trump_tweet_data["content"].apply(word_tokenize))
len(trump_tokens)

43352

Using nltk word tokenizer to tokenize each row of the content column

- We take the column with tweets, which is 'content', and apply 'word_tokenize' to split each tweet into individual word strings. We store these lists of words in 'trump_tokens'.

We can check how many tweets (lists of words) we have by measuring the length of 'trump_tokens'

In [31]:
trump_tokens[0]

['Be',
 'sure',
 'to',
 'tune',
 'in',
 'and',
 'watch',
 'Donald',
 'Trump',
 'on',
 'Late',
 'Night',
 'with',
 'David',
 'Letterman',
 'as',
 'he',
 'presents',
 'the',
 'Top',
 'Ten',
 'List',
 'tonight',
 '!']

To get a clearer idea of how the tweets have been split into words, we look at the first tokenized tweet. This helps us see the result of the tokenization process

- We can see there are 43352 sentences in the content column
- Each sentence has its own array of words

In [32]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, trump_tokens)

# Model Training: Phase 1 Pre-Processing

- Here we start preparing our tweets to build a language model. We decide on using sequences up to three words long (that's what 'n = 3'). When the value of everygram is specified the function create word sequences of length less than or equal to the specified values. The padded_everygram_pipeline function helps us format these tweets correctly for our language model.

- This function adds padding to the start and end of each tweet to handle the beginnings and endings of each setnence to keep context properly.

- The 'train_data' variable will be used for training, while 'padded_sents' contains our processed tweets.

## train_data
- It is a series of everygram sequences derived from the input text, In this case we using n = 3 which will create sequences of all lenghts less than or equal to n to calculate probabliities.

- As the intuitive name everygram suggests, it contains all the n-grams of length less than or equal to the specified values of n.

In this case it will create all unigrams, bigrams and trigram possible for each sentence
eg:

    ('<s>',),
    ('<s>', '<s>'),
    ('<s>', '<s>', 'Be'),
    ('<s>',), ('<s>', 'Be'),
    ('<s>', 'Be', 'sure'),
    ('Be',),
    ('Be', 'sure'),
    ('Be', 'sure', 'to'),
    ('sure',),
    ('sure', 'to'),
    ('sure', 'to', 'tune'),
    ('to',),
    ('to', 'tune'),
    ('to', 'tune', 'in'),
    ('tune',),
    ('tune', 'in'),
    ('tune', 'in', 'and'),
    ('in',), ('in', 'and'),
    ('in', 'and', 'watch'),
    ('and',), ('and', 'watch'),
    ('and', 'watch', 'Donald'),
    ('watch',),
    ('watch', 'Donald'),
    ('watch', 'Donald', 'Trump')

  and so on

## 'padded_sents'
 - contains our tweets after they've been processed. Each one starts and ends with markers like 's' and '/s', signaling the beginning and end of sentences.

 - This detailed structure helps our model understand context better.

 eg:
    's',
    's',
    'Be',
    'sure',
    'to',
    'tune',
    'in',
    'and',
    'watch',
    'Donald',
    'Trump',
    'on',
    'Late',
    'Night',
    'with',
    'David',
    'Letterman',
    'as',
    'he',
    'presents',
    'the',
    'Top',
    'Ten',
    'List',
    'tonight',
    '!',
    '/s',
    '/s',
    's',
    's',
    'Donald',
    'Trump',
    'will',

Initialize our language model using the Maximum Likelihood Estimation (MLE) method with 'n' (the number of words to consider, which we set to 3). Then, train the model using our prepared 'train_data' to learn from the tweet sequences.



In [33]:
trump_model = MLE(n)
trump_model.fit(train_data, padded_sents)

# Model Training: Phase 2 Training

- We utilize MLE here to estimate the parameters of our language model.
- the estimation method calculates the probability of observing the data given a set of parameters.
- We employ MLE as it helps in determining the most likely parameters that make the observed data most probable.

- 'fit' method trains 'trump_model' using 'train_data' and 'padded_sents', adapting the model to learn from the training dataset.


# Generating Setences

As the name De-tokenizer suggest, It helps us turn lists of words back into full sentences, making them easy to read.

In [34]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

# Instance of detokenizer
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

- The generate_sent function returns new sentences by choosing words one by one, aiming for a maximum length of up to n words per sentence.

- The function takes in
    - a model,
    - max number of words and
    - a random number distribtution seed,
  
  which has been set to a default value 42 if the user provides no input for the seed.

- The function works on the word pair probabilities and language patterns extracted from Trump's tweets during the model training phase.

- The function skips special markers 's' and '/s' that usually indicate the start and end of sentences. The later marks the end of the sentence and once we a '/s' we break and return a compelete sentence.

- Once we have all our words, we join them into one complete sentence using the detokenize function and return it.

- This way, each new sentence generated mirrors the linguistic style observed in the original dataset of tweets.

# Generated Sentence Outputs

In [35]:
generate_sent(trump_model, num_words=6)

'for 200 years . Thank you'

- generate_sent(trump_model, num_words=4)

- Using the num_words as 6 for a short sentence
- As we did not specify any seed, the randon_seed will take the default value of 42

 ### Generated Sentence
-  'for 200 years . Thank you'

In [36]:
generate_sent(trump_model, num_words=10, random_seed=56)

'with his family and many thousands of energy and excitement'

- generate_sent(trump_model, num_words=10, random_seed=56)


 ### Generated Sentence
-  'with his family and many thousands of energy and excitement'



In [37]:
generate_sent(trump_model, num_words=20, random_seed=112)

'Trump_Ireland graces over 500 acres fronting 2.5 miles . Extraordinary! http: //bit.ly/jyfGP3 #celebrityapprenticefinale'

- generate_sent(trump_model, num_words=20, random_seed=112)

We use the generate random functio nwith the same model but with the length of sentence specified as 20 and random numbers seed as 112

 ### Generated Sentence
- Trump_Ireland graces over 500 acres fronting 2.5 miles . Extraordinary! http: //bit.ly/jyfGP3 #celebrityapprenticefinale

# Limitations

- If we specify a length too long the model will generate a sentece that might not make sence after a while, becuase the model tries to keep the context of words to a sentence, Also, in general most sentences are not very long.

- The model works better with short sentence lengths

- For the same random seed the model will keep generating same sequence of words.